# Neural Theorem Prover using pandas and Pytorch

## 1. Symbolic Unificaiton using pandas DataFrame
- Load Files(Config, KG, Rule Template, ...)
- Generate Meta Tables(Rule Structure, KG index, ...)
- Run Backward Chaining and generate batch 

## 2. NTP Model Training with PyTorch
- Define Model Structure using PyTorch
- Define Foward Function 
- Training Model

## 3. Extract Rules from Trained Embedding Vectors
- Matching Rule templates with Embedding vectors 
- Extract Induced Rules

### Import packages

In [1]:
#custom functions
from util.fileUtils import load_conf, load_from_file, create_directory
from ntp.prover import backward_chaining
from preprocess.dataPreprocessing import data_filter, padding, proof_path_dataset
from preprocess.dataPreprocessing import convert_list_to_tensor, negative_samplig

import random
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pprint import pprint
from torch.utils.data import DataLoader
from datetime import datetime, timedelta

# to print pandas dataframe
from IPython.display import display
pd.set_option('display.max_columns', 50)

random.seed(1337)
torch.manual_seed(1337)
torch.cuda.manual_seed_all(1337)

### Load Config File
- dataname options:
    - example_8 / kinship / umls / nations
  
- parameters config:
    - data
        - kg : 지식 그래프 파일 위치
        - templates : 규칙 템플릿 파일 위치
    - meta
        - result_dir : 학습 결과인 규칙 파일이 저장될 디렉터리
        - result_file : 저장될 규칙 파일 이름
    - training
        - num_epochs : 학습 epoch 횟수
        - report_interval : 학습시 log를 report할 iteration 주기
        - batch_size : mini batch size
        - neg_per_pos : positive data 하나당 생성될 negative data수
        - learning_rate : learning rate
        - shuffle(True/False) : epoch마다 training data를 random shuffle할지 여부
    - model
        - embedding size : symbol의 vector representation size
        - l2 : weight decay rate (L2 penalty)
        - drop_porb : dropout probability
        - init(True/False) : 
            - True - xavier uniform distribution으로 embedding matrix 초기화  
            - Flase - 평균0 분산1인 normal distribution으로 embedding matrix 초기화

In [2]:
data_name = 'example_8'

In [3]:
config = load_conf(f"./config/{data_name}.conf")

embedding_size = config['model']['embedding_size']
drop_prob = config['model']['drop_prob']
weight_decay = config['model']['l2']
epochs = config['training']['num_epochs']
report_interver_epoch = config['training']['report_interval']
learning_rate = config['training']['learning_rate']
neg_per_pos = config['training']['neg_per_pos']
batch_size = config['training']['batch_size']
init = config['model']['init']
result_dir = config['meta']['result_dir']
result_file = config['meta']['result_file']
shuffle = config['training']['shuffle']

config

{'data': {'kg': './data/example_8.txt', 'templates': './data/example_8.nlt'},
 'meta': {'result_dir': './out/example_8',
  'result_file': '/example_8_rule.tsv'},
 'training': {'num_epochs': 300,
  'report_interval': 10,
  'batch_size': 2,
  'neg_per_pos': 1,
  'learning_rate': 0.001,
  'shuffle': True},
 'model': {'embedding_size': 100,
  'l2': 0.0001,
  'drop_prob': 0.05,
  'init': True}}

### Load Data Files using pandas
- KG : Knowledge Graph file with triple form
- Query : query with triple form

In [4]:
KG = pd.read_csv(config['data']['kg'], sep='\t', names=['subj','pred','obj'])
Query = pd.read_csv(config['data']['kg'], sep='\t', names=['subj','pred','obj'])

In [5]:
KG = KG[['pred', 'subj', 'obj']]
KG.head()

,pred,subj,obj
0,nationality,BART,USA
1,birthPlace,BART,NEWYORK
2,locatedIn,NEWYORK,USA
3,hasFather,BART,HOMER
4,hasGrandfather,BART,ABE


In [6]:
Query = Query[['pred', 'subj', 'obj']]
Query.head()

,pred,subj,obj
0,nationality,BART,USA
1,birthPlace,BART,NEWYORK
2,locatedIn,NEWYORK,USA
3,hasFather,BART,HOMER
4,hasGrandfather,BART,ABE


In [7]:
entity_list = sorted(set(KG.subj.values).union(set(KG.obj.values)))
len(entity_list)

6

In [8]:
start = datetime.now()

#KG index dictionary initializing
KG_index = {}
for entity in entity_list:
    KG_index[entity] = {'subj':[], 'obj':[]}
    
subj_entities = KG['subj'].tolist()
obj_entities = KG['obj'].tolist()

#KG index dictionary generation
for i in range(len(KG)):
    KG_index[subj_entities[i]]['subj'] = KG_index.get(subj_entities[i]).get('subj')+[i]
    KG_index[obj_entities[i]]['obj'] = KG_index.get(obj_entities[i]).get('obj')+[i]

end = datetime.now() 
print('converting time : ', end-start)

converting time :  0:00:00.000528


In [9]:
KG_index

{'ABE': {'subj': [], 'obj': [4, 5]},
 'BART': {'subj': [0, 1, 3, 4, 7], 'obj': [6]},
 'HOMER': {'subj': [5], 'obj': [3]},
 'LISA': {'subj': [6], 'obj': [7]},
 'NEWYORK': {'subj': [2], 'obj': [1]},
 'USA': {'subj': [], 'obj': [0, 2]}}

### Load Rule template

In [10]:
rules, max_atom = load_from_file(config['data']['templates'])
rules

[[('#1', 'X', 'Y'), ('#2', 'X', 'Z'), ('#3', 'Z', 'Y'), 2],
 [('#1', 'X', 'Y'), ('#2', 'Y', 'X'), 2]]

In [11]:
rule_structure = pd.DataFrame(list(map(lambda x : [{atom[1]: 'subj', atom[2]: 'obj'} for atom in x[:-1]], rules)))
rule_structure['rule_number'] = [i for i in range(len(rules))]
rule_structure

,0,1,2,rule_number
0,"{'X': 'subj', 'Y': 'obj'}","{'X': 'subj', 'Z': 'obj'}","{'Z': 'subj', 'Y': 'obj'}",0
1,"{'X': 'subj', 'Y': 'obj'}","{'Y': 'subj', 'X': 'obj'}",None,1


### Generate Dictionary from KG & Query data

In [12]:
KG_predicate_list = sorted(set(KG.pred.values).union(set(Query.pred.values)))

rule_pred_list = []
for i, rule in enumerate(rules):
    # iterate rule components
    for r in rule[:-1]:
        #iterate augmnet number
        for j in range(rule[-1]):
            suffix = '_' + str(i) + '_' + str(j)
            rule_pred_list.append(r[0]+suffix)
            
predicate_list = sorted(set(KG_predicate_list).union(set(rule_pred_list)))

In [13]:
id2sym_dict = {}
sym2id_dict = {}
sym2id_dict['UNK'] = 0
sym2id_dict['PAD'] = 1
id2sym_dict[0] = 'UNK'
id2sym_dict[1] = 'PAD'

for i, p in enumerate(predicate_list):
    sym2id_dict[p] = i+2
    id2sym_dict[i+2] = p

In [14]:
sym2id_dict

{'UNK': 0,
 'PAD': 1,
 '#1_0_0': 2,
 '#1_0_1': 3,
 '#1_1_0': 4,
 '#1_1_1': 5,
 '#2_0_0': 6,
 '#2_0_1': 7,
 '#2_1_0': 8,
 '#2_1_1': 9,
 '#3_0_0': 10,
 '#3_0_1': 11,
 'birthPlace': 12,
 'hasFather': 13,
 'hasGrandfather': 14,
 'hasParent': 15,
 'locatedIn': 16,
 'nationality': 17,
 'sibling': 18}

### Run Backward Chaining

#### 1. unification
- goal: query (e.g. nationality BART USA)
- rule: rule template (e.g. #1(X,Y) :- #2(X,Z), #3(Z,Y))

- 주어진 rule template의 conclusion과 query를 unify
    - unify된 트리플은 다음과 같이 `rule component substitution`에 key를 rule component(e.g. #1(X,Y))로  
        value를 unified triples(dataframe)으로 저장   
    
        #1(X, Y) :
    
            |     pred    | subj | obj |
            |-------------|------|-----|
            | nationality | BART | USA |
    
    - conclusion의 X,Y와 같은 variable에 대하여 unify된 트리플을 참조하여 `variable substitution`에 binding  
        - (e.g. `variable substitution` = X : [BART], Y: [USA]) 




- 앞서 binding된 variable을 참조하여 각 rule body에 맞는 트리플을 unify
    - #1(X,Y)를 통해 binding된 X에 대한 `variable substitution`을 참조하여 #2(X,Z)와 같은 body에 트리플을 unify하는 작업을 수행
        - #2(X,Z)의 경우에는 `variable substitution`을 참조하여 X(BART)가 subject인 트리플을 찾아 unify   
    - unify된 트리플은 다음과 같이 `rule component substitution`에 key를 rule component(e.g. #2(X,Y))로  
        value를 unified triples(dataframe)으로 저장   
           
       #2(X, Z) :

            |     pred     | subj | obj     |
            |--------------|------|---------|
            | placeOfBirth | BART | NEWYORK |
            | hasFather    | BART | HOMMER  |    
        
    - 규칙 body의 Z와 같은 variable에 대하여 unify된 트리플을 참조하여 `variable substitution`에 binding  
        - (e.g. `variable substitution` = Z : [NEWYORK, HOMMER], X : [BART], Y: [USA])
    
#### 2. proof path completion
- rule template을 분석하여 인접한 rule component간의 common variable 도출 
- common variable을 기준으로 unified triple을 join하여 proof path 생성

In [15]:
relation_path, rule_template_path, max_path, unify_dict = backward_chaining(Query, KG, KG_index, 
                                                                        rules, rule_structure, 
                                                                        sym2id_dict)

complete generating proof paths! : 8/8


### Data filtering
- proof path가 없는 데이터 제거

In [16]:
relation_path = list(filter(data_filter, relation_path))
rule_template_path = list(filter(data_filter, rule_template_path))

### Negative sampling

In [17]:
KG_relation = set(KG['pred'])
augment_num = rules[0][-1]

for idx, (relation, rule_template) in enumerate(zip(relation_path, rule_template_path)):
    if (idx+1) % 100 == 0:
        print('negative sampling... : '+str(idx+1)+'/'+str(len(relation_path)))
    elif (idx+1) == len(relation_path):
        print('complete negative sampling! : '+str(idx+1)+'/'+str(len(relation_path)))
    pos_neg_relation_path = []
    pos_neg_rule_template = []
    for rel, rule in zip(relation, rule_template):
        rel_result, rule_result = negative_samplig(rel, rule, KG_relation, augment_num, 
                                                   neg_per_pos, sym2id_dict, id2sym_dict, unify_dict)
        pos_neg_relation_path.append(rel_result)
        pos_neg_rule_template.append(rule_result)
    relation_path[idx] = tuple(pos_neg_relation_path)
    rule_template_path[idx] = tuple(pos_neg_rule_template)

complete negative sampling! : 4/4


### Padding
- rule의 최대 구성 요소 수와 최대 proof path 수로 모든 proof path padding

In [18]:
relation_path, rule_template_path = padding(relation_path, rule_template_path, rules, max_path, 
                                            max_atom, neg_per_pos, sym2id_dict)

### Create Batch Generator

In [19]:
relation_tensor = convert_list_to_tensor(relation_path)
rule_template_tensor = convert_list_to_tensor(rule_template_path)
answer = [1]
for j in range(neg_per_pos):
    answer += [0]
answer = torch.tensor(answer, dtype=torch.float32)
augment_num = rules[0][-1]

In [20]:
dataset = proof_path_dataset(relation_tensor, rule_template_tensor, answer)
batch_generator = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

### Define Model Architecture

In [21]:
class NTP(nn.Module):
    
    def __init__(self, vocab_size, embedding_size, max_path, dropout):
        super(NTP, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.embedding_matrix = nn.Embedding(self.vocab_size, self.embedding_size, 
                                             padding_idx = sym2id_dict['PAD'])
        self.max_path = max_path
        self.dropout = nn.Dropout(dropout)
        
    def RBF_kernel(self, embed_rule_template_path, embed_relation_path):
        L2_norm = torch.sqrt((embed_relation_path - embed_rule_template_path).pow(2).sum(4))
        similarity = torch.exp(-L2_norm/2)
        return similarity
    
    def calculate_sim_avg(self, rule_template_path, relation_path):

        embed_rule_template_path = self.embedding_matrix(rule_template_path)
        embed_relation_path = self.embedding_matrix(relation_path)
                                                         
        embed_rule_template_path = self.dropout(embed_rule_template_path)
        embed_relation_path = self.dropout(embed_relation_path)
        sims=self.RBF_kernel(embed_rule_template_path, embed_relation_path)
        avg_sims = torch.mean(sims, 3)
        return avg_sims
        
        
    def forward(self, rule_template_path, relation_path):
        
        avg_sims = self.calculate_sim_avg(rule_template_path, relation_path)
        max_sims = torch.max(avg_sims, axis=2)[0]
        min_sims = torch.min(max_sims.view(-1, self.max_path), axis=1)[0]
        
        return min_sims

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vocab_size = len(sym2id_dict)
ntp = NTP(vocab_size, embedding_size, max_path, drop_prob)
ntp.to(device)

NTP(
  (embedding_matrix): Embedding(19, 100, padding_idx=1)
  (dropout): Dropout(p=0.05, inplace=False)
)

In [23]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)
if init:
    ntp.apply(initialize_weights); 

### Train Relation Embedding

In [24]:
optimizer = torch.optim.Adam(ntp.parameters(), lr = learning_rate, weight_decay = weight_decay)
criterion = torch.nn.BCELoss()

ntp.train()
epoch_loss = 0
start_time = datetime.now()
estart_time = datetime.now()
for epoch in range(1, epochs+1):
    for relation_path, rule_template_path, label in batch_generator:
        optimizer.zero_grad()
        
        label = torch.flatten(label).to(device)
        
        y_hat = ntp.forward(rule_template_path.to(device), relation_path.to(device))
        
        loss = criterion(y_hat, label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    if epoch%report_interver_epoch == 0:
        end_time = datetime.now()
        print(f'Epoch Time: {end_time-estart_time} \tEpoch: {epoch}', end='')
        print(f'\tTotal Loss: {epoch_loss/epoch:.3f} \tCurrent Loss: {loss.item():.3f}')
        estart_time = end_time

end_time = datetime.now()
print('\nTotal Training Time : ', end_time-start_time)

Epoch Time: 0:00:00.129117 	Epoch: 10	Total Loss: 1.392 	Current Loss: 0.691
Epoch Time: 0:00:00.015880 	Epoch: 20	Total Loss: 1.352 	Current Loss: 0.619
Epoch Time: 0:00:00.017425 	Epoch: 30	Total Loss: 1.308 	Current Loss: 0.565
Epoch Time: 0:00:00.016141 	Epoch: 40	Total Loss: 1.264 	Current Loss: 0.536
Epoch Time: 0:00:00.017765 	Epoch: 50	Total Loss: 1.221 	Current Loss: 0.534
Epoch Time: 0:00:00.013196 	Epoch: 60	Total Loss: 1.179 	Current Loss: 0.459
Epoch Time: 0:00:00.012218 	Epoch: 70	Total Loss: 1.139 	Current Loss: 0.444
Epoch Time: 0:00:00.012362 	Epoch: 80	Total Loss: 1.101 	Current Loss: 0.377
Epoch Time: 0:00:00.012720 	Epoch: 90	Total Loss: 1.065 	Current Loss: 0.352
Epoch Time: 0:00:00.014070 	Epoch: 100	Total Loss: 1.030 	Current Loss: 0.372
Epoch Time: 0:00:00.012618 	Epoch: 110	Total Loss: 0.999 	Current Loss: 0.370
Epoch Time: 0:00:00.012330 	Epoch: 120	Total Loss: 0.971 	Current Loss: 0.343
Epoch Time: 0:00:00.012635 	Epoch: 130	Total Loss: 0.946 	Current Loss: 0

### Write rule(result) file

In [25]:
def representation_match(x, emb):
    dist = torch.torch.nn.functional.pairwise_distance(x, emb)
    sim = torch.exp(-dist)
    return sim

In [26]:
#get trained embedding matrix
for i in enumerate(ntp.parameters()):
    print(i[1])
    embeddings = i[1]

Parameter containing:
tensor([[ 1.7536e-11, -7.4200e-07, -7.2798e-13,  ..., -1.4443e-07,
          3.5233e-08,  1.3768e-15],
        [ 2.1466e-13, -8.0051e-18, -1.2674e-15,  ...,  2.8015e-04,
         -7.4563e-19,  1.9365e-15],
        [ 1.2429e-01,  1.4468e-01,  3.2746e-02,  ...,  2.0227e-01,
          1.2036e-01, -2.7266e-01],
        ...,
        [ 9.6702e-02,  8.7594e-02,  8.2490e-02,  ..., -1.6670e-01,
          1.5226e-01,  4.6534e-02],
        [-3.6866e-02,  6.5228e-03,  1.3013e-04,  ..., -4.0019e-07,
          3.5717e-02, -2.8456e-02],
        [-1.4886e-02, -1.7472e-02,  3.3357e-02,  ..., -3.3902e-02,
          6.2160e-02,  2.1144e-02]], requires_grad=True)


In [27]:
#get parameterized rule template
rule_templates = {}
idx_rule_templates = {}
for rule_number, template in enumerate(rules):
    result_template_key = []
    ids_result_template_value = []
    ids_result_template_values = []
    for i in range(len(template)-1):
        rule_element=(f'p{int(template[i][0][1])-1}_{rule_number}', template[i][1], template[i][2])       
        result_template_key.append(rule_element)
        rule_element = ()

    for aug in range(template[-1]):
        for j in range(len(template)-1):
            ids_result_template_value.append([sym2id_dict[template[j][0]+'_'+str(rule_number)+'_'+
                                                           str(aug)], template[j][1], template[j][2]])
        ids_result_template_values.append(ids_result_template_value)
        ids_result_template_value = []
    idx_rule_templates[tuple(result_template_key)] = ids_result_template_values

In [28]:
#get rule instance & write rule file

# 자기 자신을 masking하기 위한 rule template의 relation index생성
masking_index = []
for key, rules in idx_rule_templates.items():
    for rule in rules:
        for element in rule:
            masking_index.append(element[0])
            
create_directory(result_dir)
with open(result_dir + result_file, 'w') as f:
    for key, rules in idx_rule_templates.items():
        result = []
        f.write(str(key)+'\n')
        for rule in rules:
            relation_similarities = []
            rule_result = []
            for element in rule:
                masking_index = masking_index+[element[0]]+[0, 1]
                x = ntp.embedding_matrix(torch.tensor([element[0]]).to(device))
                match = representation_match(x, embeddings)
                match[masking_index] = 0
                top_k = torch.topk(match, 1)
                rule_result.append(id2sym_dict[top_k.indices.item()]+'('+element[1]+','+element[2]+')')
                relation_similarities.append(match[top_k.indices])
            confidence_score = str(min(relation_similarities).item())

            head = rule_result[0]
            body = rule_result[1:]
            
            result.append((round(min(relation_similarities).item(), 6), head + ' :- ' +", ".join(body)+'\n'))
            result.sort(reverse = True)
        for score, rule in result:
            f.write(str(score)+ '\t' + rule)
        f.write('\n')